<a href="https://colab.research.google.com/github/flydragon2018/Watchlist/blob/master/tf_refuse6_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Sep  7 13:10:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
ls

drive/  sample_data/


In [4]:
!unzip -q  drive/"My Drive"/tf_refuse.zip  

In [ ]:
pwd

'/content'

In [ ]:
cd /content/tf_refuse

/content/tf_refuse


In [ ]:
from tensorflow.keras.applications import InceptionResNetV2,NASNetLarge,EfficientNetB7 
#height , width = 150 , 150

##conv_base = InceptionResNetV2(weights='imagenet',

height , width = 331,331
conv_base = NASNetLarge(weights='imagenet',
                  include_top=False,
                  input_shape=(height,width,3))
conv_base.summary()


343613440/343610240 [==============================] - 3s 0us/step
Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 165, 165, 96) 0           stem_bn1[0][0]                   
__________________________

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os ,shutil

In [ ]:
##图像目录
dir_train = 'data/train'
dir_test = 'data/test'
## 图像文件名 和 标签
dataframe_train = pd.read_csv('data/train.csv')
dataframe_test = pd.DataFrame({ 'filename':['{}.jpg'.format(i) for i in range(506)] })
## dataframe 中图像文件名 和 标签 对应的列名
x_col = 'filename'
y_col = 'label'
##求类别个数 和 类别字典
classes = list(set(dataframe_train.iloc[:,1]))  #所有类别
classes_num = len(classes)

batch_size = 20
from keras import models
from keras import layers

In [ ]:
model = models.Sequential()
model.add(conv_base)  #在卷积基的基础上添加分类器
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.75))
model.add(layers.Dense(classes_num,activation='softmax'))
# model.summary()
##冻结直到某一层的所有层  conv2d_200及其之后的层可训练
conv_base.trainable = True
set_trainable =False
for layer in conv_base.layers:    #迭代访问网络层的方法
    if layer.name == 'conv2d_200':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
model.summary()

##使用学习率较低的RMSPROP优化器
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Functional)          (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_2 (Flatten)          (None, 487872)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               62447744  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 774       
Total params: 147,365,336
Trainable params: 62,448,518
Non-trainable params: 84,916,818
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
mycallback = [
    ModelCheckpoint(filepath='./ckpt/best_model.h5',  # 文件夹要提前自建
                    monitor='val_acc',
                    verbose=1,
                    mode='auto',
                    save_best_only=True),
    TensorBoard(log_dir='./logs/{}'.format('train'),
                histogram_freq=1,
                write_graph=True,
                update_freq='epoch'),
   
]

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)   #注意  不能增强验证数据

split_ind = int(dataframe_train.shape[0]*0.8)     #划分训练集和验证集的分隔下标
train_generator = train_datagen.flow_from_dataframe(dataframe_train[:split_ind],        #用train_datagen 有数据增强
                                                    dir_train,
                                                    x_col,y_col,
                                                    target_size=(height,width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    # seed = 666,
                                                    classes=classes)
validation_generator = test_datagen.flow_from_dataframe(dataframe_train[split_ind:],   # 用test_datagen 没数据增强
                                                    dir_train,
                                                    x_col,y_col,
                                                    target_size=(height,width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    # seed=666,
                                                    classes=classes)
test_generator = test_datagen.flow_from_dataframe(dataframe_test,
                                                  dir_test,
                                                  x_col,None,
                                                  batch_size=batch_size,
                                                  target_size=(height,width),
                                                  class_mode=None,       #用于预测的数据必须 class_mode=None 和 y_col = None
                                                  shuffle=False)         #并且shuffle=False
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['acc'])
history = model.fit(train_generator,
                              epochs=200,
                              validation_data=validation_generator,callbacks=mycallback)

Found 1616 validated image filenames belonging to 6 classes.
Found 405 validated image filenames belonging to 6 classes.
Found 506 validated image filenames.
Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.8006 - acc: 0.7760
Epoch 00001: val_acc improved from -inf to 0.85926, saving model to ./ckpt/best_model.h5
81/81 [==============================] - 95s 1s/step - loss: 0.8006 - acc: 0.7760 - val_loss: 0.7701 - val_acc: 0.8593
Epoch 2/200
81/81 [==============================] - ETA: 0s - loss: 0.8043 - acc: 0.7543
Epoch 00002: val_acc did not improve from 0.85926
81/81 [==============================] - 82s 1s/step - loss: 0.8043 - acc: 0.7543 - val_loss: 0.7185 - val_acc: 0.8593
Epoch 3/200
81/81 [==============================] - ETA: 0s - loss: 0.8639 - acc: 0.7772
Epoch 00003: val_acc improved from 0.85926 to 0.86420, saving model to ./ckpt/best_model.h5
81/81 [==============================] - 87s 1s/step - loss: 0.8639 - acc: 0.7772 - val_loss: 0.6272 - 

In [ ]:
##绘制结果
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)
plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and Validationg accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()
##评估模型
val_loss,val_acc = model.evaluate(validation_generator)
print('val acc:',val_acc)



In [ ]:
##应用在新图片
test_generator.reset()  # predict_generator之前必须reset()
# plt.imshow(next(test_generator)[0])
pred = model.predict_generator(test_generator)
predicted_class_indices=np.argmax(pred,axis=1)
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
# filenames=test_generator.filenames   #返回测试集图像的文件名序列
results=pd.DataFrame({"filename":range(506), "label":predictions})
results.to_csv("results.csv",index=False,header=False)
model.save('model_98.h5')

In [ ]:
!cp *.csv /content/drive/"My Drive"